In [1]:
import re
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
###user_pay表处理

# def SplitTimeStr(time_stamp):
#     arr = time_stamp.split(' ')
#     time = arr[1].split(':')[0]
#     return time

# def SplitDateStr(time_stamp):
#     arr = time_stamp.split(' ')
#     date = arr[0]
#     return date

#统计每家店每个时间段的消费次数
#定义表格式
#'7/1/2015到2016.10.30（除去2015.12.12）
time_index=pd.date_range(start='6/25/2015', end='11/1/2016', freq='H')
user_pay_count_df = pd.DataFrame(data=None, index=None, columns=time_index, dtype=None, copy=False)
user_pay_count_df.insert(0,'shop_id',None)
user_pay_count_df=user_pay_count_df.astype(np.str)
user_pay_count_df.columns = user_pay_count_df.columns.astype(np.str)
user_pay_count_df

#（69674110,3）
user_pay_reader = pd.read_csv("./data/dataset/dataset/user_pay.txt",
                         encoding='UTF-8',header=None,
                        names=['user_id','shop_id','time_stamp'],chunksize=100000)
index = 0;
for chunk in user_pay_reader:
#     chunk['date'] = chunk['time_stamp'].apply(SplitDateStr)
#     chunk['time'] = chunk['time_stamp'].apply(SplitTimeStr)
#     print(chunk['date'])
#     print(chunk['time'])
#     print(index)
    grouped = chunk.groupby(['shop_id','time_stamp'])
    #计数
    count_chunk = grouped.count()
    count_chunk.reset_index(inplace=True)
    count_chunk.rename(columns={'user_id':'count'}, inplace = True)
    count_pivot = count_chunk.pivot(index='shop_id', columns='time_stamp', values='count')
    count_pivot.reset_index(drop=True)
    count_pivot.columns.name = None
    count_pivot.insert(0,'shop_id',count_pivot.index)
    count_pivot.reset_index(drop=True, inplace=True)
    #合并
    user_pay_count_df = user_pay_count_df.append(count_pivot,ignore_index=True)
#     index = index + 1
#     if index == 3:
#         break

user_pay_count_df = user_pay_count_df.fillna(0)
user_pay_count_df = user_pay_count_df.astype(np.int)
user_pay_count_df

,2015-06-25 00:00:00,2015-06-25 01:00:00,2015-06-25 02:00:00,2015-06-25 03:00:00,2015-06-25 04:00:00,2015-06-25 05:00:00,2015-06-25 06:00:00,2015-06-25 07:00:00,2015-06-25 08:00:00,2015-06-25 09:00:00,...,2016-10-31 16:00:00,2016-10-31 17:00:00,2016-10-31 18:00:00,2016-10-31 19:00:00,2016-10-31 20:00:00,2016-10-31 21:00:00,2016-10-31 22:00:00,2016-10-31 23:00:00,2016-11-01 00:00:00,shop_id
0,0,0,0,0,0,0,0,0,0,0,...,3,6,2,0,0,0,0,0,0,1097
1,0,0,0,0,0,0,0,0,0,0,...,2,3,6,10,6,2,0,0,0,1862
2,0,0,0,0,0,0,0,0,0,0,...,7,16,9,4,2,8,3,4,0,1912
3,0,0,0,0,0,0,0,0,0,0,...,21,31,19,18,14,10,0,0,0,90
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,2,2,0,0,1,0,727
5,0,0,0,0,0,0,0,0,0,0,...,5,1,2,4,2,1,2,4,0,1912
6,0,0,0,0,0,0,0,0,0,0,...,0,2,1,4,5,4,3,4,0,727
7,0,0,0,0,0,0,0,0,0,0,...,3,15,26,8,8,3,0,0,0,1110
8,0,0,0,0,0,0,0,0,0,0,...,0,0,2,5,3,1,1,0,0,1780
9,0,0,0,0,0,0,0,0,0,0,...,11,9,6,7,4,3,0,0,0,1894


In [2]:
#count_pivot中有的而user_pay_count_df中没有
#print(list(set(count_pivot.columns).difference(set(user_pay_count_df.columns))))

In [3]:
user_pay_count_df = user_pay_count_df.sort_values('shop_id')
user_pay_count_df = user_pay_count_df.reset_index(drop=True)
m = user_pay_count_df.shape[0]
#通shop_id相加
current_shop_id = user_pay_count_df.loc[0, 'shop_id']
current_shop_topi = 0
for i in range(1,m):
    #print('left = %d, right = %d'%(user_pay_count_df.ix[i]['shop_id'], current_shop_id))
    if user_pay_count_df.loc[i, 'shop_id'] == current_shop_id:
        user_pay_count_df.ix[current_shop_topi] = user_pay_count_df.loc[current_shop_topi][:] + user_pay_count_df.loc[i][:]
        user_pay_count_df.iloc[current_shop_topi]['shop_id'] = current_shop_id
        #print("x = %d, y = %d, r = %d"%(user_pay_count_df.loc[current_shop_topi]['2016-10-31 18:00:00'], user_pay_count_df.loc[current_shop_topi]['2016-10-31 18:00:00'], user_pay_count_df.loc[i]['2016-10-31 18:00:00']))
    else:
        current_shop_id = user_pay_count_df.loc[i, 'shop_id']
        current_shop_topi = i

#去重
user_pay_count_df = user_pay_count_df.drop_duplicates(subset='shop_id', keep='first')
user_pay_count_df

,2015-06-25 00:00:00,2015-06-25 01:00:00,2015-06-25 02:00:00,2015-06-25 03:00:00,2015-06-25 04:00:00,2015-06-25 05:00:00,2015-06-25 06:00:00,2015-06-25 07:00:00,2015-06-25 08:00:00,2015-06-25 09:00:00,...,2016-10-31 16:00:00,2016-10-31 17:00:00,2016-10-31 18:00:00,2016-10-31 19:00:00,2016-10-31 20:00:00,2016-10-31 21:00:00,2016-10-31 22:00:00,2016-10-31 23:00:00,2016-11-01 00:00:00,shop_id
0,0,0,0,0,0,0,0,0,0,0,...,4,13,17,27,22,11,5,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,5,15,5,13,11,4,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,5,9,9,8,5,4,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,14,9,17,17,20,3,0,0,0,4
5,0,0,0,0,0,0,0,0,0,0,...,16,24,26,15,35,26,10,0,0,5
7,0,0,0,0,0,0,0,0,0,0,...,17,20,6,2,3,1,0,0,0,6
8,0,0,0,0,0,0,0,0,0,0,...,4,3,6,7,6,10,3,0,0,7
10,0,0,0,0,0,0,0,0,0,0,...,3,1,2,6,4,2,0,0,0,8
11,0,0,0,0,0,0,0,0,0,0,...,13,32,43,32,26,1,0,0,0,9
13,0,0,0,0,0,0,0,0,0,0,...,15,12,23,8,16,5,0,0,0,10


In [4]:
user_pay_count_df = user_pay_count_df.reset_index(drop=True)
user_pay_count_df.index = user_pay_count_df['shop_id']

In [5]:
user_pay_count_df = user_pay_count_df.drop('shop_id',axis=1)
user_pay_count_df = user_pay_count_df.drop('2016-11-01 00:00:00',axis=1)

In [6]:
user_pay_count_df

,2015-06-25 00:00:00,2015-06-25 01:00:00,2015-06-25 02:00:00,2015-06-25 03:00:00,2015-06-25 04:00:00,2015-06-25 05:00:00,2015-06-25 06:00:00,2015-06-25 07:00:00,2015-06-25 08:00:00,2015-06-25 09:00:00,...,2016-10-31 14:00:00,2016-10-31 15:00:00,2016-10-31 16:00:00,2016-10-31 17:00:00,2016-10-31 18:00:00,2016-10-31 19:00:00,2016-10-31 20:00:00,2016-10-31 21:00:00,2016-10-31 22:00:00,2016-10-31 23:00:00
shop_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,6,9,4,13,17,27,22,11,5,1
2,0,0,0,0,0,0,0,0,0,0,...,6,3,5,15,5,13,11,4,0,0
3,0,0,0,0,0,0,0,0,0,0,...,7,6,5,9,9,8,5,4,0,0
4,0,0,0,0,0,0,0,0,0,0,...,3,5,14,9,17,17,20,3,0,0
5,0,0,0,0,0,0,0,0,0,0,...,3,27,16,24,26,15,35,26,10,0
6,0,0,0,0,0,0,0,0,0,0,...,1,13,17,20,6,2,3,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,8,0,4,3,6,7,6,10,3,0
8,0,0,0,0,0,0,0,0,0,0,...,4,2,3,1,2,6,4,2,0,0
9,0,0,0,0,0,0,0,0,0,0,...,8,10,13,32,43,32,26,1,0,0


In [7]:
user_pay_count_df.to_csv("./data/dataset/dataset/user_pay_count.txt", sep='\t', index=True, encoding='UTF-8')